In [1]:
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
tqdm.pandas()

## Nettoyage du DF

In [52]:
# Open du fichier JSON
with open('spots_grimpe.json', 'r') as file:
    data = json.load(file)

In [53]:
# JSON TO DF
df = pd.DataFrame(data['data'])

In [54]:
# Suppression des colonnes
df = df.drop(columns= ["ref_topo", "equipement", "topo_num"])

In [55]:
# Supression du tiret situé à la fin
def sup_tiret(row):
    return row.rstrip("-")


df["type_escalade"] = df["type_escalade"].apply(sup_tiret)
df["exposition"] = df["exposition"].apply(sup_tiret)
df["public"] = df["public"].apply(sup_tiret)

In [56]:
# Création de liste à la place d'une chaîne de caractéres
def transfo_liste(chaine:str)-> list:
    liste_genres = []
    for genre in chaine.split("-"):
        liste_genres.append(genre.strip())
    return liste_genres

df["type_escalade"] = df["type_escalade"].apply(transfo_liste)
df["exposition"] = df["exposition"].apply(transfo_liste)
df["public"] = df["public"].apply(transfo_liste)
df["saison"] = df["saison"].apply(transfo_liste)

In [57]:
# Remplacer "hautniveau" par "haut niveau" dans les listes
df['public'] = df['public'].apply(lambda lst: ['haut niveau' if x == 'hautniveau' else x for x in lst])

In [58]:
# Dictionnaire de mappage
mapping_orientation = {
    "S": "Sud",
    "N": "Nord",
    "W": "Ouest",
    "E": "Est",
    "SW": "Sud-Ouest",
    "SE": "Sud-Est",
    "NW": "Nord-Ouest",
    "NE": "Nord-Est"
}

# Appliquer le mappage aux listes dans la colonne "exposition"
df["exposition"] = df["exposition"].apply(lambda lst: [mapping_orientation.get(el, el) for el in lst])

In [60]:
# Dictionnaire de mappage
mapping_mois = {
    "1": "Janvier",
    "2": "Février",
    "3": "Mars",
    "4": "Avril",
    "5": "Mai",
    "6": "Juin",
    "7": "Juillet",
    "8": "Août",
    "9": "Septembre",
    "10": "Octobre",
    "11": "Novembre",
    "12": "Decembre"

}

# Appliquer le mappage aux listes dans la colonne "saison"
df["saison"] = df["saison"].apply(lambda lst: [mapping_mois.get(el, el) for el in lst])

In [61]:
# Suppression des () et des - puis remplacer les espaces par des tirets pour l'url
df['url'] = df['nom'].str.replace('(', '').str.replace(')', '').str.replace(' - ', '-').str.replace(' ', '-')

/var/folders/fk/s9b7316s21b6krxxntv4pzzm0000gn/T/ipykernel_1485/3093565319.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['url'] = df['nom'].str.replace('(', '').str.replace(')', '').str.replace(' - ', '-').str.replace(' ', '-')


## Scrapping

In [72]:
# Définition des en-têtes
headers = {"httpapiaccesstoken": "M4rvBxc4M7kqqdtXPDvFEYm9"}
data = {}

# Parcours des IDs avec une barre de progression
for id in tqdm(range(1, 3656), desc="Fetching data"):
    entry = {}  # Dictionnaire pour l'ID actuel
    
    # URL pour route_figures
    url_level = f"https://api.oblyk.org/api/v1/public/crags/{id}/route_figures.json"
    response_level = requests.get(url_level, headers=headers)

    if response_level.status_code == 200:
        try:
            dic_level_response = response_level.json()
            if dic_level_response:
                # Utilisation de get() pour éviter les erreurs si une clé est manquante
                entry["route_count"] = dic_level_response.get("route_count")
                
                # Accès sécurisé aux éléments imbriqués
                grade = dic_level_response.get("grade", {})
                min_grade = grade.get("min", {})
                crag_route = min_grade.get("crag_route")
                if crag_route:  # Vérifiez si crag_route n'est pas None
                    crag = crag_route.get("crag", {})
                    entry["name"] = crag.get("name", "Nom non disponible")
                else:
                    entry["name"] = "Nom non disponible"
                
                entry["climbing_types"] = dic_level_response.get("climbing_types", [])
                entry["levels"] = dic_level_response.get("levels", [])
            else:
                print(f"Aucune donnée trouvée pour ID {id}")
        except ValueError:
            print(f"Invalid JSON for ID {id}")
    
    # URL pour les infos principales
    url_info = f"https://api.oblyk.org/api/v1/public/crags/{id}.json"
    response_info = requests.get(url_info, headers=headers)

    if response_info.status_code == 200:
        try:
            dic_info_response = response_info.json()
            if dic_info_response:
                # Accès sécurisé aux éléments imbriqués
                entry["north"] = dic_info_response.get("north")
                entry["north_east"] = dic_info_response.get("north_east")
                entry["east"] = dic_info_response.get("east")
                entry["south_east"] = dic_info_response.get("south_east")
                entry["south"] = dic_info_response.get("south")
                entry["south_west"] = dic_info_response.get("south_west")
                entry["west"] = dic_info_response.get("west")
                entry["north_west"] = dic_info_response.get("north_west")
                entry["summer"] = dic_info_response.get("summer")
                entry["autumn"] = dic_info_response.get("autumn")
                entry["winter"] = dic_info_response.get("winter")
                entry["spring"] = dic_info_response.get("spring")
                entry["latitude"] = dic_info_response.get("latitude")
                entry["longitude"] = dic_info_response.get("longitude")
                entry["region"] = dic_info_response.get("region")
                entry["rocks"] = dic_info_response.get("rocks")
                entry["photo"] = (
                    dic_info_response.get("photo", {})
                    .get("attachments", {})
                    .get("picture", {})
                    .get("variant_path", "Photo non disponible")
                )
            else:
                print(f"Aucune donnée trouvée pour ID {id}")
        except ValueError:
            print(f"Invalid JSON for ID {id}")
    
    # Ajouter l'ID et les informations associées au dictionnaire principal
    data[id] = entry

# Vérification du résultat
print(f"Nombre total d'entrées collectées : {len(data)}")


Fetching data: 100%|██████████| 3655/3655 [17:52<00:00,  3.41it/s] 

Nombre total d'entrées collectées : 3655


In [88]:
df = pd.DataFrame(data)

In [89]:
df = df.T

In [90]:
# Export CSV
df.to_csv("spots_grimpe.csv", index=False)


## Analyse

In [91]:
# Import CSV
df = pd.read_csv("spots_grimpe.csv")

In [92]:
df

,route_count,name,climbing_types,levels,north,north_east,east,south_east,south,south_west,...,north_west,summer,autumn,winter,spring,latitude,longitude,region,rocks,photo
0,20.0,Montiroche,"{'sport_climbing': 20, 'bouldering': 0, 'multi...","{'1a': 0, '1b': 0, '1c': 0, '2a': 0, '2b': 0, ...",False,False,False,False,False,False,...,False,False,False,False,False,48.034770,6.569101,Vosges,['migmatite'],https://storage.oblyk.org/cdn-cgi/image/:varia...
1,33.0,St Amé - Petit rocher,"{'sport_climbing': 33, 'bouldering': 0, 'multi...","{'1a': 0, '1b': 0, '1c': 0, '2a': 0, '2b': 0, ...",False,False,False,False,False,False,...,False,False,False,False,False,48.029512,6.663420,Vosges,['granite'],https://storage.oblyk.org/cdn-cgi/image/:varia...
2,19.0,Les rangs de mars,"{'sport_climbing': 19, 'bouldering': 0, 'multi...","{'1a': 0, '1b': 0, '1c': 0, '2a': 0, '2b': 0, ...",False,False,False,False,False,False,...,False,False,False,False,False,44.604847,5.015727,Drôme,['limestone'],https://storage.oblyk.org/cdn-cgi/image/:varia...
3,11.0,Label,"{'sport_climbing': 11, 'bouldering': 0, 'multi...","{'1a': 0, '1b': 0, '1c': 0, '2a': 0, '2b': 0, ...",False,False,False,False,False,False,...,False,False,False,False,False,44.602225,5.171185,Drôme,['limestone'],https://storage.oblyk.org/cdn-cgi/image/:varia...
4,68.0,Trou de la lune,"{'sport_climbing': 61, 'bouldering': 0, 'multi...","{'1a': 0, '1b': 0, '1c': 0, '2a': 0, '2b': 0, ...",False,False,False,False,False,False,...,False,False,True,False,True,46.837016,5.715881,Jura,['limestone'],https://storage.oblyk.org/cdn-cgi/image/:varia...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3650,4.0,Castel Valley,"{'sport_climbing': 0, 'bouldering': 0, 'multi_...","{'1a': 0, '1b': 0, '1c': 0, '2a': 0, '2b': 0, ...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,38.651600,-109.368000,Grand County,[],NaN
3651,3.0,Le Rocher Marre,"{'sport_climbing': 0, 'bouldering': 0, 'multi_...","{'1a': 0, '1b': 0, '1c': 0, '2a': 0, '2b': 0, ...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,43.578384,2.967457,Hérault,[],NaN
3652,1.0,La Petite Suisse,"{'sport_climbing': 0, 'bouldering': 0, 'multi_...","{'1a': 0, '1b': 0, '1c': 0, '2a': 0, '2b': 0, ...",False,False,False,False,False,False,...,False,NaN,NaN,NaN,NaN,43.578500,2.968120,Hérault,[],NaN
3653,1.0,La Pointe à Pistre,"{'sport_climbing': 0, 'bouldering': 0, 'multi_...","{'1a': 0, '1b': 0, '1c': 0, '2a': 0, '2b': 0, ...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,43.578848,2.968634,Hérault,[],NaN
